In [1]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Embedding
import pickle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [2]:
pickle_file = "../processed_data/20200224-202814"

with open(pickle_file, "rb") as f:
    X, y, tokenizer, len_vocab = pickle.load(f) 

In [8]:
number_of_embeddings = 50
LSTM_units = 256
dropout = 0.1

# define model
model = Sequential()
model.add(Embedding(len_vocab, number_of_embeddings, input_length=2))
if dropout > 0:
    model.add(Dropout(dropout))
model.add(GRU(LSTM_units))#, return_sequences=True, return_state=False))
#model.add(GRU(LSTM_units, return_sequences=True, return_state=True))
model.add(Dense(len_vocab, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 2, 50)             371700    
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 50)             0         
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               235776    
_________________________________________________________________
dense_2 (Dense)              (None, 7434)              1910538   
Total params: 2,518,014
Trainable params: 2,518,014
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# compile network
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Write callback section

In [10]:
filepath = "../best_callbacks/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [checkpoint, early_stopping]

In [11]:
len_seq = X.shape[0]
def build_model(X, y, v_split=0.2, number_of_epochs=10, verbose=1):
    model.fit(X, y, validation_split=v_split, epochs=num_epochs, verbose=verbose)
print(X.shape[0])

124846


In [12]:
# fit network
num_epochs = 10
#batch_n = 50
validation_split = 0.33

build_model(X, y, v_split=validation_split, number_of_epochs=num_epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 83646 samples, validate on 41200 samples
Epoch 1/10
12256/83646 [===>..........................] - ETA: 1:07 - loss: 6.8215 - acc: 0.0321

KeyboardInterrupt: 

Function that will convert from the embedded numbers to text with a number of predictions.

In [ ]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # pre-pad sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
    return in_text

In [ ]:
max_length = 3

test = "there is"

keras_embedder = tokenizer

generate_seq(model, keras_embedder, max_length-1, test, 10)

In [ ]:
test1 = "My wife"
test2 = "My husband"
test3 = "My friend"
test4 = "My fiance"
test5 = "My (22M)"
test6 = "My girlfriend"
test7 = "My boyfriend"
test8 = "My partner"
test9 = "My (23F)"
test10 = "My spouse"

length = 25

print(generate_seq(model, keras_embedder, max_length-1, test1, length),"\n")
print(generate_seq(model, keras_embedder, max_length-1, test2, length),"\n")
print(generate_seq(model, keras_embedder, max_length-1, test3, length),"\n")
print(generate_seq(model, keras_embedder, max_length-1, test4, length),"\n")
print(generate_seq(model, keras_embedder, max_length-1, test5, length),"\n")
print(generate_seq(model, keras_embedder, max_length-1, test6, length),"\n")
print(generate_seq(model, keras_embedder, max_length-1, test7, length),"\n")
print(generate_seq(model, keras_embedder, max_length-1, test8, length),"\n")
print(generate_seq(model, keras_embedder, max_length-1, test9, length),"\n")
print(generate_seq(model, keras_embedder, max_length-1, test10, length),"\n")
